**SMS 3.1（政策文本版）源代码及其操作**

In [1]:
#以下代码块用于支持所有操作的环境搭建
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from paddlenlp import Taskflow
import os, re
import pandas as pd
import numpy as np
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec

d:\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\python37\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
d:\python37\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
#装载算法
# seg = Taskflow('word_segmentation', user_dict = './userdict/userdict-paddle.txt', mode = 'accurate', batch_size=32)
tag = Taskflow('pos_tagging', user_dict ='./dict/user_dict.txt', mode='accurate', batch_size=32)

In [5]:
if __name__ == '__main__':#传入参数
    stop_path = './dict/hit_stopwords.txt'#停用词位置
    dict_path = './dict/user_dict.txt'#用户自定义词典
    data_path = './按照阶段合并/第二阶段/'#选择要处理的具体阶段
    seg_str_path = './S第二阶段seg_str/'#选择要处理的具体阶段对应的分词存储位置
    edge_path = tfidf_path = './tfidf/temp_out2.xlsx'
    temp_out = './各阶段共词网络/'
    model_path = 'model'

In [3]:
#以下代码块为语料预处理做支撑
def LL_stop(stop_path): # 装载哈工大中文停用词表
    f_stop = open(stop_path,'r', encoding='utf-8') 
    sw = [line.strip() for line in f_stop]  # 首次遍历停用词表
    f_stop.close()  #清空缓存 
    return sw

In [6]:
#以下代码块用于分词，并返回适用于word2vec列表数据接口（但是非常占内存！！！）
filter_list = ['v','n','an']

filter_result = []

for id in os.listdir(data_path):

    pre_cor = open(data_path + id, 'r', encoding='utf-8')
    cor = pre_cor.read()
    re_tag = [item for item in tag(cor)]
    pre_cor.close()#关闭文件
    for word,flag in re_tag:
        if flag in filter_list:
            filter_result.append(word)  
LL_fenci = [term for term in filter_result if term not in LL_stop(stop_path) and len(term) > 1]


In [7]:
#以下代码块用于将LL-fenci结果存储至Word2Vec模型训练中，每个循环只可运行一次。
cor = ' '.join(LL_fenci)
print(cor, file=open(model_path + '/' + 'train_data.txt', 'a', encoding='utf-8'))

In [8]:
#以下代码块读取LL_fenci内容，并将其存储为可供二次查询的信息
def write_LL_fenci():
    filter_list = ['v','n','an']

    for id in os.listdir(data_path):
        
        filter_result = []

        pre_cor = open(data_path + id, 'r', encoding='utf-8')
        cor = pre_cor.read()
        re_tag = [item for item in tag(cor)]
        pre_cor.close()#关闭文件
        for word,flag in re_tag:
            if flag in filter_list:
                filter_result.append(word)  
    # LL_fenci = [term for term in filter_result if term not in LL_stop(stop_path) and len(term) > 1]
        write_LL_fenci = [term for term in filter_result if term not in LL_stop(stop_path) and len(term) > 1]
        ee = open(seg_str_path + '/' + id, 'w', encoding='utf-8')#输出结果
        ee.write(' '.join(write_LL_fenci))
        ee.close()
write_LL_fenci()

In [9]:
#以下代码块用于全部文档级别的关键词tf-idf值的计算与输出；除非更新数据，否则请勿独立运行(现已删除索引列)
corpus = []
for id in os.listdir(seg_str_path):
    contend = open(seg_str_path + id,'r',encoding='utf-8').read()
    corpus.append(contend)

def tf_idf():
    vectorizer = CountVectorizer()#将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
    transformer = TfidfTransformer()#统计每个词语的tf-idf权值
    tfidf = transformer.fit_transform(vectorizer.fit_transform(corpus))

    word = vectorizer.get_feature_names_out()
    weight = tfidf.toarray()

    data = {'word': word,'tfidf': weight.sum(axis=0).tolist()}

    df = pd.DataFrame(data)
    sorted_df = df.sort_values(by="tfidf", ascending = False).iloc[0:100]#对dataframe格式的tfidf矩阵以“by=tfidf”值降序排列，并“iloc[0:60]”输出前60个的值
    # print(sorted_df)
    sorted_df.to_excel(tfidf_path,index=False)#将sorted_df结果写入到xlsx
    # excel = pd.read_excel(tfidf_path, index_col= None)
    # excel.drop(excel.columns[0], axis=1, inplace=True)
    
tf_idf()

`******可修改过滤词性**********`

In [12]:
#共现词矩阵模块（1）：以下代码块通过函数Get_check_cor(data_path)用于生成所有文档"句子级别"的分词结果，并以二维列表形式返回结果以支持二次遍历
tag_filter = ['v', 'n','an']#可在此处修改词性过滤条件

data_array = []

for id in os.listdir(data_path):
    res_list = []
    raw_contend = open(data_path + '/' + id, 'r', encoding = 'utf-8').read()
    raw_contend = str(re.split('；|。|：|\n',raw_contend))

    seg_result = [item for item in tag(raw_contend)]

    for word,flag in seg_result:
        if flag in tag_filter:
            res_list.append(word)
    data_array.append([item for item in res_list if item not in LL_stop(stop_path) and len(item) > 1])

In [11]:
共现词矩阵模块（2）：以下代码准备从新的路径载入tf-idf_excel中的数据；以下代码块用于初始化矩阵和填入共现数据，注意：对角线数据为0
def get_edge(edge_path):
    edge = []
    raw_edge = pd.read_excel(edge_path, sheet_name=0,usecols=[0])
    raw_edge = raw_edge.values.tolist()

    for i in raw_edge:
        edge.append(i[0])

    return edge

edge = get_edge(edge_path)


In [13]:
#以下代码块用于生成共现词网络
def matrix(edge):
    
    matrix = [['' for j in range(len(edge) + 1)] for i in range(len(edge) + 1)]
    
    matrix[0][1:] = np.array(edge)

    matrix = list(map(list, zip(*matrix)))

    matrix[0][1:] = np.array(edge)
    return matrix

matrix = matrix(edge)

def count_matrix(matrix, data_array):

    for row in range(1, len(matrix)):
        for col in range(1, len(matrix)):
            if matrix[0][row] == matrix[col][0]:
                matrix[col][row] = str(0)

            else:
                counter = 0

                for ech in data_array:
                    if matrix[0][row] in ech and matrix[col][0] in ech:
                        counter += 1
                    else:
                        continue
                matrix[col][row] = str(counter)

    data1 = pd.DataFrame(matrix)
    data1.to_csv(temp_out +'共词.csv', index=0, columns=None, encoding='utf_8_sig', header=False)
count_matrix(matrix, data_array)

`*****请确认模型数据，否则不要运行以下所有代码！！！*******`

In [ ]:
#word2vec：除非更新模型，否则请不要运行这个代码块
sentences=word2vec.Text8Corpus(model_path + '/' + 'train_data.txt')
model= Word2Vec(sentences, sg=1, vector_size=100,window=5,min_count=3, negative=5, hs=1)
model.save(model_path + '/' + 'word2vec_model1.bin')
model.wv.save_word2vec_format(model_path + '/' + 'word2vec_model2.txt', binary = "False") 

In [ ]:
#Kmeans(1)：装载word2vec语言模型并进行Kmeans聚类
model = Word2Vec.load(model_path + '/' + 'word2vec_model1.bin')

keys = model.wv.key_to_index.keys()

# print(keys)
# print(type(keys))
# print(list(keys)[0])


In [ ]:
#输出PCA降维后的特征
edge = list(set([line.strip() for line in open('./各阶段共词网络/关键词汇总.txt','r',encoding='utf-8').readlines()]))

wordvector = [model.wv[key] for key in keys if key in edge]
edge_wordvector = [word for word in keys if word in edge]

col_num = []#声明一个列表
for num in range(1, 101):
    col_num.append(str(num)+'维')

df_wordvector = pd.DataFrame(wordvector,index=list(edge_wordvector), columns=list(col_num))

df_wordvector.to_csv('你肯定知道我是干啥的.csv',encoding='utf_8_sig')

版权所有 (c) 2023 [**@Pluckism**]

特此授予任何获取本软件及其相关文档文件(以下简称“软件”)的人免费使用的权利，包括但不限于使用、复制、修改、合并、发布、分发、再授权和/或出售软件的权利，并允许被许可人按照上述条件使用软件。

在所有软件副本或实质部分中包含以上版权声明和本许可声明。

本软件按“原样”提供，不附带任何形式的明示或暗示的保证，包括但不限于适销性保证。在任何情况下，作者或版权持有人均不对因使用软件或无法使用软件而导致的任何直接或间接损失承担责任。


Copyright (c) 2023 [**@Pluckism**]

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

